In [ ]:
import numpy as np
import pandas as pd
from collections import OrderedDict
import random

from sklearn.model_selection import train_test_split

import tensorflow as tf
# tf.config.run_functions_eagerly(True)
import keras
from keras import layers
from keras import regularizers
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
import keras
from keras.models import load_model
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, Conv2DTranspose, concatenate
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

In [ ]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Globals

In [ ]:
SEED = 100

In [ ]:
AMIGOS_PATH = '/content/drive/MyDrive/Amigos/'

In [ ]:
target_df = pd.read_csv(AMIGOS_PATH + 'target.csv', index_col = 0)
target_df = target_df.drop(['Valence','Arousal'], axis = 1)

target_df[target_df['Dominance'] <= 4.5] = 0
target_df[target_df['Dominance'] > 4.5] = 1

target_df

,Dominance
Subjects,
1,1.0
1,1.0
1,0.0
1,1.0
1,0.0
...,...
40,0.0
40,0.0
40,1.0


In [ ]:
data = np.array(pd.read_csv('/content/drive/MyDrive/Amigos/Amigos_extractedfeatures.csv', index_col = 0))
target = np.array(target_df)

In [ ]:
data.shape, target.shape

((800, 85), (800, 1))

In [ ]:
# Make dictionary with each subject

data_dict = dict()
target_dict = dict()

for idx in range(40):
    data_dict[idx] = data[20*idx : 20*(idx+1)]
    target_dict[idx] = target[20*idx : 20*(idx+1)]

len(data_dict), len(target_dict)

(40, 40)

In [ ]:
subjects = list(range(40))
random.Random(SEED).shuffle(subjects)
print(subjects)
# data points > # clients
# Using client size = 6, # clients = 3

[35, 0, 12, 24, 18, 28, 15, 19, 36, 26, 30, 34, 13, 16, 37, 4, 5, 33, 39, 31, 10, 6, 20, 21, 1, 8, 14, 23, 2, 3, 17, 7, 32, 27, 22, 25, 11, 38, 29, 9]


In [ ]:
train_clients = subjects[:33]
test_clients = subjects[33:]
train_clients, test_clients

([35,
  0,
  12,
  24,
  18,
  28,
  15,
  19,
  36,
  26,
  30,
  34,
  13,
  16,
  37,
  4,
  5,
  33,
  39,
  31,
  10,
  6,
  20,
  21,
  1,
  8,
  14,
  23,
  2,
  3,
  17,
  7,
  32],
 [27, 22, 25, 11, 38, 29, 9])

In [ ]:
def create_clientwise_data(client_ids):
    x = tf.concat([data_dict[id] for id in client_ids], 0)
    y = tf.concat([target_dict[id] for id in client_ids], 0)
    return x, y

In [ ]:
X_train, Y_train = create_clientwise_data(train_clients)
X_test, Y_test = create_clientwise_data(test_clients)

X_train.shape, Y_train.shape, X_test.shape, Y_test.shape

(TensorShape([660, 85]),
 TensorShape([660, 1]),
 TensorShape([140, 85]),
 TensorShape([140, 1]))

In [ ]:
input_shape = 85
ann = tf.keras.models.Sequential()

#Adding First Hidden Layer
ann.add(tf.keras.layers.Dense(32, input_dim=85, activation="relu"))
#Adding Second Hidden Layer
ann.add(tf.keras.layers.Dense(units=16, activation="relu"))
#Adding Output Layer
ann.add(tf.keras.layers.Dense(1, activation="sigmoid"))

#Compiling ANN
ann.compile(optimizer="adam",loss="binary_crossentropy",metrics=['binary_accuracy'])

In [ ]:
ann.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 32)                2752      
                                                                 
 dense_7 (Dense)             (None, 16)                528       
                                                                 
 dense_8 (Dense)             (None, 1)                 17        
                                                                 
Total params: 3,297
Trainable params: 3,297
Non-trainable params: 0
_________________________________________________________________


In [ ]:
ann.fit(X_train, Y_train,epochs = 200)

Epoch 1/200
21/21 [==============================] - 1s 2ms/step - loss: 0.8909 - binary_accuracy: 0.5561
Epoch 2/200
21/21 [==============================] - 0s 2ms/step - loss: 0.8157 - binary_accuracy: 0.5697
Epoch 3/200
21/21 [==============================] - 0s 2ms/step - loss: 0.7901 - binary_accuracy: 0.5439
Epoch 4/200
21/21 [==============================] - 0s 2ms/step - loss: 0.7766 - binary_accuracy: 0.5455
Epoch 5/200
21/21 [==============================] - 0s 2ms/step - loss: 0.7427 - binary_accuracy: 0.5621
Epoch 6/200
21/21 [==============================] - 0s 2ms/step - loss: 0.7014 - binary_accuracy: 0.5939
Epoch 7/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6878 - binary_accuracy: 0.5924
Epoch 8/200
21/21 [==============================] - 0s 2ms/step - loss: 0.7079 - binary_accuracy: 0.5591
Epoch 9/200
21/21 [==============================] - 0s 2ms/step - loss: 0.6761 - binary_accuracy: 0.6030
Epoch 10/200
21/21 [==========================

# Evaluation

In [ ]:
ann.evaluate(x=X_test, y=Y_test)

5/5 [==============================] - 0s 3ms/step - loss: 3.6335 - binary_accuracy: 0.5500


[3.633511781692505, 0.550000011920929]

In [ ]:
np.sum(Y_test) / len(Y_test)

0.7071428571428572